In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False    # 解决无法显示符号的问题
# sns.set(font='SimHei', font_scale=0.8)        # 解决Seaborn中文显示问题
sns.set_style("white", {"font.sans-serif": ["SimHei"]})

# 1. Load all csv data

In [27]:
dir_path = "~/projects/healthman/"
df_meta = pd.read_csv(f"{dir_path}/analysis/tableOnePlus-final.csv", 
                      na_values=pd.NA, encoding="utf-8", index_col=[0])
df_meta

,item_name,count,mean,std,min,25%,50%,75%,max,TYPE
v000760,身高,179773.0,165.805661,8.325735,0.0,159.6,165.5,171.9,199.8,NUMBER
v000761,体重,179786.0,64.576750,12.648138,18.6,54.7,63.2,72.9,180.5,NUMBER
v002266,腹围,43013.0,82.744077,10.197139,50.0,75.0,83.0,90.0,144.0,NUMBER
v000763,心率,183995.0,81.690193,12.640567,35.0,73.0,81.0,89.0,164.0,NUMBER
v000762,血压(收缩压),183994.0,121.111683,17.171112,62.0,109.0,119.0,132.0,241.0,NUMBER
...,...,...,...,...,...,...,...,...,...,...
LUNG.ProliferativeLesion,肺增殖灶,110011.0,0.430730,0.495181,0.0,0.0,0.0,1.0,1.0,CATEGORY
LUNG.SolidNodule,肺实性结节,110011.0,0.017707,0.131886,0.0,0.0,0.0,0.0,1.0,CATEGORY
LUNG.SmallNodule,肺小结节,110011.0,0.163456,0.369783,0.0,0.0,0.0,0.0,1.0,CATEGORY
LUNG.Nodule,肺结节,110011.0,0.354701,0.478425,0.0,0.0,0.0,1.0,1.0,CATEGORY


In [3]:
import re

# v4990
def parse_4990(x):
    if pd.isna(x):
        return np.nan
    
    string = x.upper()
    match = re.match(r'^(\d+(?:\.\d+)?)X10E(\d+)$', string)
    if match:
        base = float(match.group(1))
        exponent = int(match.group(2))    
        result = base * (10 ** exponent)
        return result
    
    return np.nan


def get_cate_val(x, top_val):
    if pd.isna(x):
        return x

    if x == top_val:
        return 0

    return 1


# if wear glass, jiaozheng
def parse_002286(x):
    if pd.isna(x):
        return np.nan

    if type(x) != str:
        return x

    pattern = r'矫正'
    result = re.search(pattern, x)
    if result:
        return 1

    return 0

def parse_0090(x):
    if pd.isna(x):
        return np.nan
    
    if  str(x)[0:2] == '阳性':
        return 1
    return 0

# return [GaoXueYa, GaoXueZhi, GaoXueTang, GanZang, JiaZhuangXian,  Others
def parse_002169(x):
    res = pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
    if pd.isna(x):
        return res

    res = pd.Series([0, 0, 0, 0, 0, 0])
    if x == "无":
        return res

    pattern = r"高血压|降压"
    result = re.search(pattern, x)
    if result:
        res[0] = 1

    pattern = r"血脂|降脂"
    result = re.search(pattern, x)
    if result:
        res[1] = 1

    pattern = r"血糖|降糖"
    result = re.search(pattern, x)
    if result:
        res[2] = 1

    pattern = r"肝"
    result = re.search(pattern, x)
    if result:
        res[3] = 1
    
    pattern =  r"优甲乐|甲状腺"
    result = re.search(pattern, x)
    if result:
        res[4] = 1


    if res.sum() == 0:
        res[5] = 1

    return res

# sleep
def parse_003193(x):
    dict_kv = {
        "好" : 4,
        "较好": 3,
        "一般" : 2,
        "较少":  1,
        "一般较少": 0,
        "少": 0,
    }
    if pd.isna(x) or x not in dict_kv:
        return np.nan

    return dict_kv[x]

def parse_5046(x):
    if pd.isna(x):
        return np.nan
    if  str(x)[0:1] == '阴' or str(x)[0:1] == "-" :
        return 0
    return 1


def parse_5047(x):
    if pd.isna(x):
        return np.nan
    
    if str(x)[0:1] == '阳' or str(x)[0:1] == "+" :
        return 1
    return 0

def parse_5048(x):
    if pd.isna(x):
        return np.nan
    if  str(x) == 'Ⅰ' or str(x) == "I" or str(x)=="1.0":
        return 1
    if  str(x) == 'Ⅱ' or str(x) == "II" or str(x)=="2.0":
        return 2
    if  str(x) == 'Ⅲ' or str(x) == "III" or str(x)=="3.0":
        return 3
    if  str(x) == 'Ⅳ' or str(x) == "IV" or str(x)=="4.0":
        return 3
    return 0

def parse_5054(x):
    if pd.isna(x) or (str(x)[0:1] == '无') or \
            (str(x)[0:1] == '空') or \
            (str(x)=="量少无法检测") or (str(x)=="/"):
        return np.nan
    if  (str(x)[0:1] == '阴') or (str(x)[0:1] == "-") or (str(x)[0:1] == "未") or (str(x) == "φ"):
        return 0
    return 1


def gen_elem_drug(s_line, input_str):
    s_line["name"] = input_str
    return s_line




In [4]:
def get_num_part_of_val(x):
    if type(x) != str:
        return x

    l_val = x.split(";")
    l_res = []
    for val in l_val:
        val = "".join(val.split("干扰"))
        val = "".join(val.split("<"))
        val = "".join(val.split(">"))
        l_res.append(val)

    try:
        x_new = np.nanmean([float(x) for x in l_res])
        return x_new
    except:
        return np.nan
    

dir_path = "/cluster/home/bqhu_jh/projects/healthman/"
df0 = pd.read_csv(f"{dir_path}/analysis/detail_2020.csv", na_values=pd.NA, encoding="utf-8")
df1 = pd.read_csv(f"{dir_path}/analysis/detail_2021.csv", na_values=pd.NA, encoding="utf-8")
df2 = pd.read_csv(f"{dir_path}/analysis/detail_2022.csv", na_values=pd.NA, encoding="utf-8")
df3 = pd.read_csv(f"{dir_path}/analysis/detail_2023.csv", na_values=pd.NA, encoding="utf-8")

df_merged = pd.concat([df0, df1, df2, df3], axis=0)
df_merged["item_id"] = [ "v%s" % x for x in df_merged["item_id"] ]
df_merged

/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/2459243432.py:23: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(f"{dir_path}/analysis/detail_2022.csv", na_values=pd.NA, encoding="utf-8")


,exam_id,serial_number,combine_item_id,combine_item_name,item_id,iterm_name,results,conclusion,unit,operation_date,review_date
0,420052500128,577862780.0,002114,内科检查,v000109,本科既往史和现患疾病,无殊,NaN,NaN,2020-12-01 8:52:47,2020-12-01 8:53:37
1,420052500128,577862781.0,002114,内科检查,v002271,肺部,两肺呼吸音清，无干湿罗音,NaN,NaN,2020-12-01 8:52:47,2020-12-01 8:53:37
2,420052500128,577862782.0,002114,内科检查,v002272,心脏,心律齐，未闻及病理性杂音,NaN,NaN,2020-12-01 8:52:47,2020-12-01 8:53:37
3,420052500128,577862783.0,002114,内科检查,v002251,腹部,腹平软，无压痛，未及明显包块，肝脾肋下未及,NaN,NaN,2020-12-01 8:52:47,2020-12-01 8:53:37
4,420052500128,580123093.0,20201201CBG098,血常规,v9045,白细胞计数,6.6,NaN,10E9/L,2020-12-09 1:23:47,2020-12-01 5:28:46
...,...,...,...,...,...,...,...,...,...,...,...
5455715,423052901081,835129438.0,001999,外科检查女(体检中心),v000091,皮肤检查,皮肤无黄染，无皮疹。,NaN,NaN,2023-06-20 9:53:34,2023-06-20 9:56:00
5455716,423052901081,835129448.0,001999,外科检查女(体检中心),v000094,四肢关节活动度,正常,NaN,NaN,2023-06-20 9:53:34,2023-06-20 9:56:00
5455717,423052901081,835129449.0,001999,外科检查女(体检中心),v002281,摇摆试验,阴性,NaN,NaN,2023-06-20 9:53:34,2023-06-20 9:56:00
5455718,423052901081,835129450.0,001999,外科检查女(体检中心),v002282,髋关节内外旋试验,无殊,NaN,NaN,2023-06-20 9:53:34,2023-06-20 9:56:00


# 2. Preprocessing

## pivot_table

In [5]:
def aggr_table(x):
    try:
        return np.nanmean([float(v) for v in x])
    except:
        return ";".join(str(v) for v in x)
    

l_colnames = list(df_meta.index) + ["v002169"]
df_merged_pvt = df_merged[df_merged["item_id"].isin(l_colnames)][["exam_id", "item_id", "results"]].\
        pivot_table(index="exam_id", columns="item_id", values="results", aggfunc=aggr_table)

df_merged_pvt

/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/3003890185.py:3: RuntimeWarning: Mean of empty slice
  return np.nanmean([float(v) for v in x])


item_id,v000048,v000079,v000082,v000083,v000084,v000085,v000086,v000087,v000090,v000091,...,v9900,v9907,v9908,v9909,v9910,v9911,v9922,v9923,v9925,vJ1000
exam_id,,,,,,,,,,,,,,,,,,,,,
401706280888,无异常,NaN,NaN,NaN,NaN,NaN,NaN,NaN,无殊,皮肤无黄染，无皮疹。,...,20.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401706280892,无异常,NaN,NaN,NaN,NaN,NaN,NaN,NaN,无殊,皮肤无黄染，无皮疹。,...,23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401706280897,无异常,NaN,NaN,NaN,NaN,NaN,NaN,NaN,无殊,皮肤无黄染，无皮疹。,...,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401707190396,无异常,无殊,无异常,阴道壁光滑,量适中，色味无明显异常。,宫颈萎缩,萎缩,未扪及异常包块。,无殊,NaN,...,7.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401707310395,无异常,NaN,NaN,NaN,NaN,NaN,NaN,NaN,无殊,NaN,...,6.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423063003063,无异常,NaN,NaN,NaN,NaN,NaN,NaN,NaN,无殊,皮肤无黄染，无皮疹。,...,NaN,NaN,NaN,NaN,NaN,NaN,阴性,阴性,阴性,阴性
423063003066,无异常,无殊,无异常,阴道壁光滑,量适中，色味无明显异常。,纳氏囊肿，宫颈轻度柱状上皮移位,大小正常。,未扪及异常包块。,无殊,皮肤无黄染，无皮疹。,...,14.1,阴性0.1,阴性0.1,阴性0.7,阴性0.1,阴性0.7,NaN,NaN,NaN,阴性
423063003067,无异常,NaN,NaN,NaN,NaN,NaN,NaN,NaN,无殊,皮肤无黄染，无皮疹。,...,NaN,NaN,NaN,NaN,NaN,NaN,阴性,阴性,阴性,阴性


## processing numeric data

In [6]:
def get_num_part_of_val(x):
    if type(x) != str:
        return x

    l_val = x.split(";")
    l_res = []
    for val in l_val:
        val = "".join(val.split("<"))
        val = "".join(val.split(">"))
        val = "".join(val.split("+"))
        val = ".".join(val.split(".."))
        val = "".join(val.split("已复核"))
        val = "".join(val.split("复查"))
        val = "".join(val.split("已复"))
        l_res.append(val)

    try:
        x_new = np.nanmean([float(x) for x in l_res])
        return x_new
    except:
        # print("\t",x)
        return np.nan

l_numeric_col = list(df_meta[df_meta["TYPE"]=="NUMBER"].index)
df_merged_num = df_merged_pvt[l_numeric_col]
for colname in l_numeric_col:
    # print("COLNAME:", colname)
    df_merged_num[colname] = [ get_num_part_of_val(x) for x in df_merged_pvt[colname] ]

df_merged_num["v003193"] = df_merged_pvt["v003193"].apply(lambda x: parse_003193(x))
df_merged_num["v4990"] = df_merged_pvt["v4990"].apply(lambda x: parse_4990(x))
    
df_merged_num

/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/808430264.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_num[colname] = [ get_num_part_of_val(x) for x in df_merged_pvt[colname] ]
/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/808430264.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_num["v003193"] = df_merged_pvt["v003193"].apply(lambda x: parse_003193(x))
/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/808430264.py:31: SettingWithCopyWarning: 
A value is trying to b

item_id,v000760,v000761,v002266,v000763,v000762,v002161,v0008,v0009,v0010,v1235,...,v4590,v4990,v3183,v4635,v4828,v4831,v4829,v9458,v9459,v9460
exam_id,,,,,,,,,,,,,,,,,,,,,
401706280888,164.5,70.7,86.0,84.0,159.0,88.0,5.60,2.12,1.06,3.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401706280892,163.0,53.3,67.0,74.0,94.0,61.0,4.97,0.49,1.76,2.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401706280897,164.0,74.0,92.0,93.0,134.0,96.0,4.56,2.11,0.76,2.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401707190396,157.0,50.8,NaN,84.0,140.0,91.0,4.21,1.94,1.29,2.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401707310395,159.5,78.8,NaN,59.0,163.0,98.0,6.13,1.44,1.66,3.74,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423063003063,155.9,85.1,NaN,87.0,121.0,80.0,4.55,1.70,0.93,2.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423063003066,162.0,52.7,67.0,65.0,94.0,55.0,4.98,0.49,2.26,2.41,...,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423063003067,165.5,51.5,NaN,83.0,106.0,62.0,4.46,0.55,2.05,2.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## processing category data

In [7]:
l_string_col = list(set(df_merged_pvt.columns)-set(l_numeric_col))

df_meta_desc = df_merged_pvt[l_string_col].describe().T
df_meta_desc["ratio"] = df_meta_desc["freq"] / df_meta_desc["count"]
df_meta_desc.sort_values(["ratio"])

,count,unique,top,freq,ratio
item_id,,,,,
v4630,6124,287,-,154,0.025147
v4773,1785,221,阳性0.01,120,0.067227
v4772,1784,510,阴性0.10,120,0.067265
v3064,1788,1306,阳性> 000.00,137,0.076622
v3001,1786,105,阴性0.34,175,0.097984
...,...,...,...,...,...
v002046,49844,5,无肿胀，无异常分泌物，导管口正常。,49838,0.99988
v002037,49883,4,正常,49877,0.99988
v002267,49886,3,无充盈,49882,0.99992


In [8]:
df_merged_str = df_merged_pvt[l_string_col].copy()
for cate_name in l_string_col:
    # cate_name = l_cate_names[0]
    df_merged_str[cate_name] = df_merged_str[cate_name].apply(
        lambda x: get_cate_val(x, df_meta_desc.loc[cate_name]['top'])
    ).astype(np.float32)


df_merged_str["v002286"] = df_merged_pvt["v002286"].apply(lambda x: parse_002286(x))
df_merged_str["v0090"]   = df_merged_pvt["v0090"].apply(lambda x: parse_0090(x))
df_merged_str["v5046"]   = df_merged_pvt["v5046"].apply(lambda x: parse_5046(x))
df_merged_str["v5047"]   = df_merged_pvt["v5047"].apply(lambda x: parse_5047(x))
df_merged_str["v5048"]   = df_merged_pvt["v5048"].apply(lambda x: parse_5048(x))
df_merged_str["v003193"] = df_merged_pvt["v003193"].apply(lambda x: parse_003193(x))
df_merged_str

/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/472892593.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_merged_str["v003193"] = df_merged_pvt["v003193"].apply(lambda x: parse_003193(x))


item_id,v002407,v0388,v4631,v4747,v9922,v0390,v4743,v002268,v4613,v000082,...,v002272,v002400,v4746,v0476,v3001,v4773,v0090,v003196,v5054,v003193
exam_id,,,,,,,,,,,,,,,,,,,,,
401706280888,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0
401706280892,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,3.0
401706280897,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,1.0,0.0,NaN,4.0
401707190396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401707310395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423063003063,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
423063003066,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,...,1.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,2.0
423063003067,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [9]:
df_merged_str[
        ["v002169.HBp","v002169.HPL","v002169.HGL","v002169.LIVER","v002169.THYROID","v002169.OTHERS"]
    ] = df_merged_pvt["v002169"].apply(lambda x: parse_002169(x))
df_merged_str

/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/1530211742.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_merged_str[
/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/1530211742.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_merged_str[
/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/1530211742.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) 

item_id,v002407,v0388,v4631,v4747,v9922,v0390,v4743,v002268,v4613,v000082,...,v0090,v003196,v5054,v003193,v002169.HBp,v002169.HPL,v002169.HGL,v002169.LIVER,v002169.THYROID,v002169.OTHERS
exam_id,,,,,,,,,,,,,,,,,,,,,
401706280888,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,1.0,1.0,NaN,2.0,1.0,0.0,0.0,0.0,0.0,0.0
401706280892,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,1.0,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0
401706280897,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,...,1.0,0.0,NaN,4.0,0.0,0.0,0.0,0.0,0.0,0.0
401707190396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401707310395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423063003063,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423063003066,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
423063003067,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_main = pd.concat([
    pd.read_csv(f"{dir_path}/analysis/main_2020.csv", index_col=["exam_id"]),
    pd.read_csv(f"{dir_path}/analysis/main_2021.csv", index_col=["exam_id"]),
    pd.read_csv(f"{dir_path}/analysis/main_2022.csv", index_col=["exam_id"]),
    pd.read_csv(f"{dir_path}/analysis/main_2023.csv", index_col=["exam_id"]),
])
df_main

,exam_date,gender,birthday,sample_id
exam_id,,,,
401707310395,2020-12-01,1,1968-01-19,Xw8iZNJucL2PKQ/bqwbTUMytDTpxZtZxWl25MNVEUgp1ug==
401804030187,2020-12-01,2,1969-03-19,sVczndQYy1MQOKM4sANUesytDTpxZtZxWly5MtVEUwt0uA==
419050901222,2020-11-22,1,1968-09-04,URESY7YR9CxPZzh1vuO9KsytDz5zZdZxWl25ONRJWg53vA==
420083102388,2020-11-06,1,1967-09-23,s/cgv9+urdphE6P/OmDD2syrDz5zZdZxWlK5ONZOUg5/tw==
402008291960,2020-11-05,2,1987-10-06,r3mnFX7WRQRVkyQV/YsEbMytBj9zZNZxVFK4MdRLVgdwtw==
...,...,...,...,...
423061201290,2023-06-14,2,1987-05-21,p4LrQbSYr9f5cCUJRSduDcypDzZ7YtZxVFK5NNZMUg5yvw==
423061400016,2023-06-14,2,1987-12-08,oKPKK3Kiu/yDkWKrlfBLWsytDTpzY9ZxVFK4M9RFVgp0vg==
423061400017,2023-06-14,2,1977-12-16,wEsTb9otCaxOB/oIXwuWr8yoDTtxY9ZxW1K4M9VLUw90ug==


In [11]:
df_table1plus = df_main.join(
                        pd.concat([df_merged_num, df_merged_str], axis=0)#[df_merged_pvt.columns]
                    , how="right")
df_table1plus

,exam_date,gender,birthday,sample_id,v000760,v000761,v002266,v000763,v000762,v002161,...,v4773,v0090,v003196,v5054,v002169.HBp,v002169.HPL,v002169.HGL,v002169.LIVER,v002169.THYROID,v002169.OTHERS
exam_id,,,,,,,,,,,,,,,,,,,,,
401706280888,2021-03-25,1,1966-06-26,aYdr0PYiYKupUF6tTVEt7MytDz5zZ9ZxWlO5N9ZLUgp3vw==,164.5,70.7,86.0,84.0,159.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401706280888,2021-03-25,1,1966-06-26,aYdr0PYiYKupUF6tTVEt7MytDz5zZ9ZxWlO5N9ZLUgp3vw==,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0
401706280892,2021-07-23,2,2000-08-04,eUtHbxYTLbIv6JPw3CgzSsytDzdxY9V4XFW5OdRJUg50tw==,163.0,53.3,67.0,74.0,94.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401706280892,2021-07-23,2,2000-08-04,eUtHbxYTLbIv6JPw3CgzSsytDzdxY9V4XFW5OdRJUg50tw==,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
401706280897,2021-11-12,1,1980-09-24,uNrG/PeBu3EpghRvEfu+rMyqDzdxZdZxVFW5ONZJUQh3uw==,164.0,74.0,92.0,93.0,134.0,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423063003067,2023-06-30,2,1996-12-28,VAuHAK38OjVG/reNpARoo8yoDztxZdZxVVO4M9ZFUwtyvg==,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
423063003078,2023-06-30,2,2000-11-04,lW3U660fStKZItX6bKWtcMyqDTpxYtV4XFW4MNRJUQl0uA==,161.7,64.2,NaN,97.0,127.0,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423063003078,2023-06-30,2,2000-11-04,lW3U660fStKZItX6bKWtcMyqDTpxYtV4XFW4MNRJUQl0uA==,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


## add columns for text-descriptions of liver/lung

In [12]:
def detect_descriptions(x, descriptions):
    if pd.isna(x):
        return x
    
    for description in descriptions.split("|"):
        if re.search(description, x):
            return 1
    return 0


def aggr_table(x):
    try:
        return np.nanmean([float(v) for v in x])
    except:
        return ";".join(str(v) for v in x)


def extract_text_from_details(df_merged, scan_items, dict_descriptions):
    df_tmp_pvt = df_merged[df_merged["combine_item_name"].\
                isin(scan_items)]\
                [["exam_id", "conclusion"]].\
                pivot_table(index="exam_id", values="conclusion", aggfunc=aggr_table)

    for k in dict_descriptions:
        v = dict_descriptions[k]
        df_tmp_pvt[v] = df_tmp_pvt["conclusion"].apply(lambda x: detect_descriptions(x, k))

    return df_tmp_pvt.reset_index()


df_table1plus

,exam_date,gender,birthday,sample_id,v000760,v000761,v002266,v000763,v000762,v002161,...,v4773,v0090,v003196,v5054,v002169.HBp,v002169.HPL,v002169.HGL,v002169.LIVER,v002169.THYROID,v002169.OTHERS
exam_id,,,,,,,,,,,,,,,,,,,,,
401706280888,2021-03-25,1,1966-06-26,aYdr0PYiYKupUF6tTVEt7MytDz5zZ9ZxWlO5N9ZLUgp3vw==,164.5,70.7,86.0,84.0,159.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401706280888,2021-03-25,1,1966-06-26,aYdr0PYiYKupUF6tTVEt7MytDz5zZ9ZxWlO5N9ZLUgp3vw==,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0
401706280892,2021-07-23,2,2000-08-04,eUtHbxYTLbIv6JPw3CgzSsytDzdxY9V4XFW5OdRJUg50tw==,163.0,53.3,67.0,74.0,94.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401706280892,2021-07-23,2,2000-08-04,eUtHbxYTLbIv6JPw3CgzSsytDzdxY9V4XFW5OdRJUg50tw==,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
401706280897,2021-11-12,1,1980-09-24,uNrG/PeBu3EpghRvEfu+rMyqDzdxZdZxVFW5ONZJUQh3uw==,164.0,74.0,92.0,93.0,134.0,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423063003067,2023-06-30,2,1996-12-28,VAuHAK38OjVG/reNpARoo8yoDztxZdZxVVO4M9ZFUwtyvg==,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
423063003078,2023-06-30,2,2000-11-04,lW3U660fStKZItX6bKWtcMyqDTpxYtV4XFW4MNRJUQl0uA==,161.7,64.2,NaN,97.0,127.0,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423063003078,2023-06-30,2,2000-11-04,lW3U660fStKZItX6bKWtcMyqDTpxYtV4XFW4MNRJUQl0uA==,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
scan_items = ["常规心电图检查"]
dict_descriptions = {
    "正常心电图": "HEART.Normal",
    "窦性心律;": "HEART.SinusRhythm",
    "T波改变": "HEART.T_change",
    "窦性心动过缓": "HEART.SinusBradycardia",
    "ST段" : "HEART.ST_change",
    "窦性心律不齐": "HEART.SinusArrhythmia",
    "电轴": "HEART.ElectricAxisDeviation",
    "左室高电压": "HEART.LVH",
    "窦性心动过速": "HEART.SinusTachycardia",
    "房颤|房性" : "HEART.AtrialFibrillation",
    "传导阻滞": "HEART.HeartBlocks",
    "早搏" : "HEART.ExtraSystole",
}
df_heart_pvt = extract_text_from_details(df_merged, scan_items, dict_descriptions)
scan_items = ["肝胆脾胰（彩超）", "肝胆脾胰+双肾（彩超）", "胸部CT平扫", "肺部HR \x01T平扫"]
dict_descriptions = {
    "脂肪肝": "LIVER.Steatosis",
    "肝囊肿|肝多发囊肿": "LIVER.HepaticCyst",
    "胆囊息肉|胆囊多发息肉": "GALLBLADDER.GallBladderPolyps",
    "肾囊肿|肾多发囊肿": "KIDNEY.RenalCyst",
}
df_liver_pvt = extract_text_from_details(df_merged, scan_items, dict_descriptions)

scan_items = ["胸部CT平扫", "肺部CT平扫", "数字化摄影(DR)", "+胸部CT平扫", "CT胸部平扫", "CT肺部HR平扫"]
dict_descriptions = {
    "磨玻璃": "LUNG.GroundGlassOpacity",
    "积液": "LUNG.Effusion",
    "纤维": "LUNG.Fibrosis",
    "钙化": "LUNG.Calcification",
    "增殖灶": "LUNG.ProliferativeLesion",
    "实性结节": "LUNG.SolidNodule",
    "小结节": "LUNG.SmallNodule",
    "结节": "LUNG.Nodule",
    "未见明显异常": "LUNG.Normal"
}
df_lung_pvt = extract_text_from_details(df_merged, scan_items, dict_descriptions)
df_merged_tableone_plus_s7 = df_table1plus.\
    merge(df_liver_pvt.drop(["conclusion"], axis=1), on="exam_id", how="left").\
    merge(df_heart_pvt.drop(["conclusion"], axis=1), on="exam_id", how="left").\
    merge(df_lung_pvt.drop(["conclusion"], axis=1), on="exam_id", how="left")
df_merged_tableone_plus_s7

/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/18043074.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean([float(v) for v in x])
/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/18043074.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean([float(v) for v in x])
/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/18043074.py:13: RuntimeWarning: Mean of empty slice
  return np.nanmean([float(v) for v in x])
/cluster/home/bqhu_jh/tmp/TMPDIR/ipykernel_1465831/1296692030.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_merged_tableone_plus_s7 = df_table1plus.\


,exam_id,exam_date,gender,birthday,sample_id,v000760,v000761,v002266,v000763,v000762,...,HEART.ExtraSystole,LUNG.GroundGlassOpacity,LUNG.Effusion,LUNG.Fibrosis,LUNG.Calcification,LUNG.ProliferativeLesion,LUNG.SolidNodule,LUNG.SmallNodule,LUNG.Nodule,LUNG.Normal
0,401706280888,2021-03-25,1,1966-06-26,aYdr0PYiYKupUF6tTVEt7MytDz5zZ9ZxWlO5N9ZLUgp3vw==,164.5,70.7,86.0,84.0,159.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,401706280888,2021-03-25,1,1966-06-26,aYdr0PYiYKupUF6tTVEt7MytDz5zZ9ZxWlO5N9ZLUgp3vw==,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,401706280892,2021-07-23,2,2000-08-04,eUtHbxYTLbIv6JPw3CgzSsytDzdxY9V4XFW5OdRJUg50tw==,163.0,53.3,67.0,74.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,401706280892,2021-07-23,2,2000-08-04,eUtHbxYTLbIv6JPw3CgzSsytDzdxY9V4XFW5OdRJUg50tw==,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,401706280897,2021-11-12,1,1980-09-24,uNrG/PeBu3EpghRvEfu+rMyqDzdxZdZxVFW5ONZJUQh3uw==,164.0,74.0,92.0,93.0,134.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462255,423063003067,2023-06-30,2,1996-12-28,VAuHAK38OjVG/reNpARoo8yoDztxZdZxVVO4M9ZFUwtyvg==,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
462256,423063003078,2023-06-30,2,2000-11-04,lW3U660fStKZItX6bKWtcMyqDTpxYtV4XFW4MNRJUQl0uA==,161.7,64.2,NaN,97.0,127.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
462257,423063003078,2023-06-30,2,2000-11-04,lW3U660fStKZItX6bKWtcMyqDTpxYtV4XFW4MNRJUQl0uA==,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
462258,423063003089,2023-06-30,2,1991-09-05,Kpeb+iPIntVHjg6gEOTo2sytDzxxZtZxVVS5ONRIUA1w1g==,162.0,54.8,72.0,95.0,133.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_table1plus = df_merged_tableone_plus_s7.copy()
df_table1plus_colnames = df_table1plus.columns
df_table1plus[["year", "month", "day"]] = pd.DataFrame({
    "year": [  x.split("-")[0] for x in df_table1plus["exam_date"] ],
    "month": [ x.split("-")[1] for x in df_table1plus["exam_date"] ],
    "day": [   x.split("-")[2] for x in df_table1plus["exam_date"] ],
})
df_table1plus["gender"] = df_table1plus["gender"].apply(lambda x: "male" if x ==1 else "female" )
df_table1plus = df_table1plus[["birthday", "year", "month", "day", "gender"] + \
                                  list(df_table1plus_colnames[4:])]
df_table1plus

# df_table1plus.to_csv(f"{dir_path}/analysis/tableOnePlusData-final.csv")


,birthday,year,month,day,gender,sample_id,v000760,v000761,v002266,v000763,...,HEART.ExtraSystole,LUNG.GroundGlassOpacity,LUNG.Effusion,LUNG.Fibrosis,LUNG.Calcification,LUNG.ProliferativeLesion,LUNG.SolidNodule,LUNG.SmallNodule,LUNG.Nodule,LUNG.Normal
0,1966-06-26,2021,03,25,male,aYdr0PYiYKupUF6tTVEt7MytDz5zZ9ZxWlO5N9ZLUgp3vw==,164.5,70.7,86.0,84.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1966-06-26,2021,03,25,male,aYdr0PYiYKupUF6tTVEt7MytDz5zZ9ZxWlO5N9ZLUgp3vw==,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-08-04,2021,07,23,female,eUtHbxYTLbIv6JPw3CgzSsytDzdxY9V4XFW5OdRJUg50tw==,163.0,53.3,67.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-08-04,2021,07,23,female,eUtHbxYTLbIv6JPw3CgzSsytDzdxY9V4XFW5OdRJUg50tw==,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1980-09-24,2021,11,12,male,uNrG/PeBu3EpghRvEfu+rMyqDzdxZdZxVFW5ONZJUQh3uw==,164.0,74.0,92.0,93.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462255,1996-12-28,2023,06,30,female,VAuHAK38OjVG/reNpARoo8yoDztxZdZxVVO4M9ZFUwtyvg==,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
462256,2000-11-04,2023,06,30,female,lW3U660fStKZItX6bKWtcMyqDTpxYtV4XFW4MNRJUQl0uA==,161.7,64.2,NaN,97.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
462257,2000-11-04,2023,06,30,female,lW3U660fStKZItX6bKWtcMyqDTpxYtV4XFW4MNRJUQl0uA==,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
462258,1991-09-05,2023,06,30,female,Kpeb+iPIntVHjg6gEOTo2sytDzxxZtZxVVS5ONRIUA1w1g==,162.0,54.8,72.0,95.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_table1plus_pvt = pd.melt(df_table1plus, 
                                id_vars=["birthday", "year", "month", "day", "gender", "sample_id"]).\
    pivot_table(index=["birthday", "year", "month", "day", "gender", "sample_id"], 
                columns="variable", values="value", aggfunc=np.nanmean
    )
df_table1plus_pvt.reset_index()[
    ["birthday", "year", "month", "day", "gender", "sample_id"] + list(df_meta.index)
].to_csv(f"{dir_path}/analysis/tableOnePlusData-final.csv")

In [30]:
df_table1plus_final = pd.read_csv(f"{dir_path}/analysis/tableOnePlusData-final.csv", index_col=[0])
df_table1plus_final

,birthday,year,month,day,gender,sample_id,v000760,v000761,v002266,v000763,...,HEART.T_change,LUNG.GroundGlassOpacity,LUNG.Effusion,LUNG.Fibrosis,LUNG.Calcification,LUNG.ProliferativeLesion,LUNG.SolidNodule,LUNG.SmallNodule,LUNG.Nodule,LUNG.Normal
0,1921-05-27,2021,6,26,male,4iqiKKzfREBB3MVNWR+UUsytDz5zY9ZxXlS5NNZKUgd1vQ==,160.9,58.1,NaN,72.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1921-05-27,2023,5,3,male,4iqiKKzfREBB3MVNWR+UUsytDz5zY9ZxXlS5NNZKUgd1vQ==,159.7,54.1,NaN,80.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1921-08-23,2022,1,25,male,QXLzAV007fmjRfOPeaKz1sytDz5zY9ZxXlS5OdZOUwt3tg==,NaN,NaN,NaN,86.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1922-12-10,2021,11,13,male,yBFQrDE47Y4nCjJ60UliQsytDz5zYtZxXle4M9VNUg53vA==,161.1,55.3,NaN,70.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1922-12-10,2022,11,10,male,yBFQrDE47Y4nCjJ60UliQsytDz5zYtZxXle4M9VNUg53vA==,160.3,58.4,NaN,60.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230741,2011-02-22,2022,7,13,female,5AZOjjqr3GQWgTQRNJFnGMytDz1yY9V4XVS5M9ZPVgp0vw==,162.6,54.7,NaN,102.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
230742,2011-04-23,2022,7,26,female,8ndCFiXVdoXsF4ZEsoQdr8ytDzpxYNV4XVS5NdZOUwt0vw==,145.5,41.7,NaN,102.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
230743,2011-10-03,2023,4,19,male,ufF3/pTqSP4aGrbGkmCai8ytDz5zZ9V4XVS4MdROVAlzvw==,156.5,51.2,NaN,78.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
230744,2012-10-17,2022,11,14,male,vHZsE+WaMR/JG+FuJJIchcytDzlxYNV4XVe4MdVKUw9/vQ==,149.0,35.3,NaN,89.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [1]:
from utils import update_liuzhong_health_check_data

df_table1plus, l_consecute_man2p, l_consecute_man3p, df_table1plus_3p_rev_month,\
                df_meta_group, rename_dict = update_liuzhong_health_check_data(
        file_data="/cluster/home/bqhu_jh/projects/healthman/analysis/tableOnePlusData-final.csv",
        file_meta="/cluster/home/bqhu_jh/projects/healthman/analysis/feature_groups_en_v2.csv"
)